**Objetivo:**   
* Preparar los datos tanto de IRAS como de SDSS, pues se han identificado múltiples observaciones para la misma posición (ra, dec)  
* Datos iniciales de IRAS= 11.444, SDSS= 329.650 (máximo valor que obtengo dado:  
  búsqueda con "WHERE subclass IN ('STARFORMING', 'STARBURST', 'AGN', 'BROADLINE')" En un total de 500.000 galaxias que es el máximo de SDSS.  
  esperando así un cruce de 1.349  
* Posteriormente en el cuaderno 2. Empezar el proceso de Crossmatching. 

In [1]:
import numpy as np
import pandas as pd
import time
import pyvo
import warnings
import astropy.units as u
import matplotlib.pyplot as plt
from datetime import date
from astropy.table import Table
from astropy.table import hstack
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
from matplotlib import pyplot as plt
from astropy import coordinates, units as u
from astroquery.xmatch import XMatch

Could not import regions, which is required for some of the functionalities of this module.


## 1. Datos para IRAS:

In [2]:
service = pyvo.dal.TAPService ('https://irsa.ipac.caltech.edu/TAP')
result = service.run_async("""
            SELECT
            galname,
            ra,
            dec,
            fnu_12,
            fnu_25,
            fnu_60,
            fnu_100,
            rat_12_25,
            rat_25_60,
            rat_60_100
            
            FROM irasgal
            ORDER BY galname ASC""")
IRAS = result.to_table()

In [3]:
IRAS = IRAS.to_pandas()

In [4]:
len(IRAS)

11444

In [5]:
IRAS[0:5]

,galname,ra,dec,fnu_12,fnu_25,fnu_60,fnu_100,rat_12_25,rat_25_60,rat_60_100
0,00000-0359,0.644752,-3.710484,0.25,0.33,1.21,2.49,0.757576,0.272727,0.485944
1,00001+0827,0.666350,8.737270,0.25,0.28,0.70,1.19,0.892857,0.400000,0.588235
2,00003+0313,0.719551,3.504221,0.25,0.32,0.52,1.12,0.781250,0.615385,0.464286
3,00003-3431,0.728076,-34.238483,1.06,0.25,0.80,2.57,4.240000,0.312500,0.311284
4,00005+2140,0.787838,21.959749,0.25,1.05,4.25,4.51,0.238095,0.247059,0.942350


In [6]:
# Cantidad de valores duplicados
valores_duplicados = IRAS.duplicated(subset='ra')
conteo_valores_duplicados = valores_duplicados.sum()
conteo_valores_duplicados

4

In [7]:
# Valores duplicados
valores_duplicados = IRAS[IRAS.duplicated(subset='ra')]['ra'].unique()
print("Valores duplicados en la columna 'ra':", valores_duplicados)

Valores duplicados en la columna 'ra': [132.58559 179.51375 185.48906 351.8132 ]


In [8]:
# Indices de los valores duplicados en la columna 'ra'
indices_duplicados = IRAS[IRAS.duplicated(subset='ra')].index
print("Índices de los valores duplicados en la columna 'ra':", indices_duplicados)

Índices de los valores duplicados en la columna 'ra': Int64Index([4105, 5929, 6215, 11228], dtype='int64')


In [9]:
# Indices de TODAS las ocurrencias de valores duplicados en la columna 'ra'
IndDuplCompletos = IRAS[IRAS['ra'].isin(valores_duplicados)].index

# Imprimir los índices de TODAS las ocurrencias de valores duplicados
print("Índices de TODAS las ocurrencias de valores duplicados en la columna 'ra':", IndDuplCompletos)

Índices de TODAS las ocurrencias de valores duplicados en la columna 'ra': Int64Index([4100, 4105, 5928, 5929, 6213, 6215, 11227, 11228], dtype='int64')


In [10]:
# Duplicado 1.1
IRAS[4100:4101]

,galname,ra,dec,fnu_12,fnu_25,fnu_60,fnu_100,rat_12_25,rat_25_60,rat_60_100
4100,08458+6638,132.585587,66.456436,0.43,0.25,0.4,1.0,1.72,0.625,0.4


In [11]:
# Duplicado 1.2
IRAS[4105:4106]

,galname,ra,dec,fnu_12,fnu_25,fnu_60,fnu_100,rat_12_25,rat_25_60,rat_60_100
4105,08463+6016,132.585587,60.086784,0.25,0.31,0.31,0.83,0.806452,1.0,0.373494


In [12]:
# Eliminar las filas con valores duplicados en la columna 'ra'
IRAS = IRAS.drop(indices_duplicados)

In [13]:
len(IRAS)

11440

## 2. Datos para SDSS:

In [14]:
query = """SELECT 
    s.specobjid, s.ra, s.dec, s.z, g.h_alpha_flux, g.h_beta_flux,
    g.oiii_5007_flux, g.nii_6548_flux, g.nii_6584_flux,
    g.sii_6717_flux, g.sii_6731_flux, g.oi_6300_flux, s.class, s.subclass

    FROM SpecObjAll AS s
    JOIN galSpecExtra AS ID ON ID.specObjID = s.specObjID
    JOIN galSpecLine AS g ON g.specObjID = s.specObjID


    WHERE subclass IN ('STARFORMING', 'STARBURST', 'AGN', 'BROADLINE')
    AND NOT h_alpha_flux <='0' AND NOT h_beta_flux <='0'
    AND NOT oiii_5007_flux <='0' AND NOT nii_6548_flux <='0'
    AND NOT nii_6584_flux <='0' AND NOT sii_6717_flux <='0' 
    AND NOT sii_6731_flux <='0' AND NOT oi_6300_flux <='0'
    AND z < 0.8

    ORDER BY specobjid ASC"""

SDSS = SDSS.query_sql(query)

C:\ProgramData\Anaconda33\lib\site-packages\astroquery\sdss\core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),


In [15]:
SDSS = SDSS.to_pandas()

In [16]:
len(SDSS)

329650

In [17]:
SDSS[0:5]

,specobjid,ra,dec,z,h_alpha_flux,h_beta_flux,oiii_5007_flux,nii_6548_flux,nii_6584_flux,sii_6717_flux,sii_6731_flux,oi_6300_flux,class,subclass
0,299489677444933632,146.71421,-1.041304,0.021222,472.59040,119.49560,68.39606,74.302520,224.11360,124.71150,84.18053,18.594930,b'GALAXY',b'STARFORMING'
1,299490502078654464,146.62857,-0.765137,0.064656,99.40005,25.64047,21.44437,17.104640,51.59155,35.33497,21.65778,8.490424,b'GALAXY',b'STARFORMING'
2,299491051364706304,146.96390,-0.545003,0.055988,181.52620,47.36650,11.85567,17.948720,54.13749,31.28298,19.16762,4.319232,b'GALAXY',b'STARBURST'
3,299491051834468352,146.63167,-0.988278,0.052654,127.30580,38.33226,35.46671,8.550285,25.78963,30.26996,25.72297,6.269853,b'GALAXY',b'STARFORMING'
4,299492700632147968,146.59272,-0.760256,0.065024,211.50180,53.18228,15.59901,25.314650,76.35482,41.63770,28.47964,7.211103,b'GALAXY',b'STARFORMING'


In [18]:
# Cantidad de valores duplicados
valores_duplicados = SDSS.duplicated(subset='ra')
conteo_valores_duplicados = valores_duplicados.sum()
conteo_valores_duplicados

19304

In [19]:
# Contar la frecuencia de cada valor en la columna "ra"
frecuencias = SDSS['ra'].value_counts()

In [20]:
# Contar el número de valores únicos
num_valores_unicos = frecuencias[frecuencias == 1].count()

# Contar el número de valores repetidos 1 vez
num_repetidos_1 = frecuencias[frecuencias == 2].count()

# Contar el número de valores repetidos 2 veces
num_repetidos_2 = frecuencias[frecuencias == 3].count()

# Contar el número de valores repetidos 3 veces
num_repetidos_3 = frecuencias[frecuencias == 4].count()

# Contar el número de valores repetidos 4 veces
num_repetidos_4 = frecuencias[frecuencias == 5].count()

# Contar el número de valores repetidos 5 veces
num_repetidos_5 = frecuencias[frecuencias == 6].count()

# Contar el número de valores repetidos 6 veces
num_repetidos_6 = frecuencias[frecuencias == 7].count()

# Contar el número de valores repetidos 7 veces
num_repetidos_7 = frecuencias[frecuencias == 8].count()

# Contar el número de valores repetidos 8 veces
num_repetidos_8 = frecuencias[frecuencias == 9].count()

# Contar el número de valores repetidos 9 veces
num_repetidos_9 = frecuencias[frecuencias == 10].count()

# Contar el número de valores repetidos 10 veces
num_repetidos_10 = frecuencias[frecuencias == 11].count()


In [21]:
print(f"Número de valores únicos: {num_valores_unicos}")
print(f"Número de valores repetidos 1 vez: {num_repetidos_1}")
print(f"Número de valores repetidos 2 veces: {num_repetidos_2}")
print(f"Número de valores repetidos 3 veces: {num_repetidos_3}")
print(f"Número de valores repetidos 4 veces: {num_repetidos_4}")
print(f"Número de valores repetidos 5 veces: {num_repetidos_5}")
print(f"Número de valores repetidos 6 veces: {num_repetidos_6}")
print(f"Número de valores repetidos 7 veces: {num_repetidos_7}")
print(f"Número de valores repetidos 8 veces: {num_repetidos_8}")
print(f"Número de valores repetidos 9 veces: {num_repetidos_9}")
print(f"Número de valores repetidos 10 veces: {num_repetidos_10}")

Número de valores únicos: 293697
Número de valores repetidos 1 vez: 14895
Número de valores repetidos 2 veces: 1239
Número de valores repetidos 3 veces: 211
Número de valores repetidos 4 veces: 242
Número de valores repetidos 5 veces: 45
Número de valores repetidos 6 veces: 14
Número de valores repetidos 7 veces: 3
Número de valores repetidos 8 veces: 0
Número de valores repetidos 9 veces: 0
Número de valores repetidos 10 veces: 0


In [22]:
a = 293697+14895+1239+211+242+45+14+3
a

310346

In [23]:
# Pre Repetidos:
b= 14895+1239+211+242+45+14+3
b

16649

In [24]:
# Pre Repetidos 2:
c= (1239*2)+(211*3)+(242*4)+(45*5)+(14*6)+(3*7)
c

4409

In [25]:
#Repetidos totales: 
# N° Repetidos * cada una de sus veces + N° una vez repetido
d= 4409+14895
d
# d son 19.304, igual a la operación "conteo_valores_duplicados"

19304

In [26]:
x= 329650-19304-293697 #Ultimos son los valores no repetidos
x
#16649 es b.

16649

In [27]:
#y = total - repetidos
y = 329650-19304
y

310346

In [28]:
# Valores duplicados
valores_duplicados = SDSS[SDSS.duplicated(subset='ra')]['ra'].unique()
print("Valores duplicados en la columna 'ra':", valores_duplicados)

Valores duplicados en la columna 'ra': [146.9639  146.94999 146.63167 ... 257.38367 258.13561 257.98823]


In [29]:
len(valores_duplicados)

16649

In [30]:
SDSS[0:10]

,specobjid,ra,dec,z,h_alpha_flux,h_beta_flux,oiii_5007_flux,nii_6548_flux,nii_6584_flux,sii_6717_flux,sii_6731_flux,oi_6300_flux,class,subclass
0,299489677444933632,146.71421,-1.041304,0.021222,472.59040,119.49560,68.39606,74.302520,224.11360,124.71150,84.18053,18.594930,b'GALAXY',b'STARFORMING'
1,299490502078654464,146.62857,-0.765137,0.064656,99.40005,25.64047,21.44437,17.104640,51.59155,35.33497,21.65778,8.490424,b'GALAXY',b'STARFORMING'
2,299491051364706304,146.96390,-0.545003,0.055988,181.52620,47.36650,11.85567,17.948720,54.13749,31.28298,19.16762,4.319232,b'GALAXY',b'STARBURST'
3,299491051834468352,146.63167,-0.988278,0.052654,127.30580,38.33226,35.46671,8.550285,25.78963,30.26996,25.72297,6.269853,b'GALAXY',b'STARFORMING'
4,299492700632147968,146.59272,-0.760256,0.065024,211.50180,53.18228,15.59901,25.314650,76.35482,41.63770,28.47964,7.211103,b'GALAXY',b'STARFORMING'
5,299492701101910016,146.96390,-0.545003,0.055981,195.51970,42.86956,16.26119,20.297290,61.22130,33.02700,24.30562,2.184580,b'GALAXY',b'STARFORMING'
6,299493250857723904,146.94999,-0.592202,0.064779,229.13250,60.80691,29.73346,24.491550,73.87218,45.46917,33.85824,7.237874,b'GALAXY',b'STARFORMING'
7,299493525265868800,146.94999,-0.592202,0.064777,233.11410,62.10917,35.72292,23.550690,71.03431,44.16328,32.56358,7.674830,b'GALAXY',b'STARFORMING'
8,299493800143775744,146.63167,-0.988278,0.052659,116.86990,36.78978,32.56529,8.366628,25.23568,24.13779,23.15360,2.733802,b'GALAXY',b'STARFORMING'
9,299494075021682688,146.62857,-0.765137,0.064589,99.23278,26.19861,17.28930,16.263180,49.05350,32.67037,18.77828,5.045829,b'GALAXY',b'STARFORMING'


In [31]:
# Indices de los valores duplicados en la columna 'ra'
indices_duplicados = SDSS[SDSS.duplicated(subset='ra')].index
print("Índices de los valores duplicados en la columna 'ra':", indices_duplicados)

Índices de los valores duplicados en la columna 'ra': Int64Index([     5,      7,      8,      9,     10,     12,     15,     18,
                20,     22,
            ...
            329098, 329099, 329100, 329102, 329103, 329116, 329285, 329438,
            329439, 329576],
           dtype='int64', length=19304)


In [32]:
indices_ra_igual_145_51241 = SDSS.index[SDSS['ra'] == 145.51241]

# Imprimir los índices
print("Índices donde el valor de 'ra' es igual a 145.51241:", indices_ra_igual_145_51241)

Índices donde el valor de 'ra' es igual a 145.51241: Int64Index([167, 170], dtype='int64')


In [33]:
# Duplicado 1.1
SDSS[167:168]

,specobjid,ra,dec,z,h_alpha_flux,h_beta_flux,oiii_5007_flux,nii_6548_flux,nii_6584_flux,sii_6717_flux,sii_6731_flux,oi_6300_flux,class,subclass
167,299595780317014016,145.51241,0.339171,0.006379,153.8173,35.59267,3.656026,13.19764,39.80714,16.52884,11.71782,1.011221,b'GALAXY',b'STARBURST'


In [34]:
# Duplicado 1.2
SDSS[170:171]

,specobjid,ra,dec,z,h_alpha_flux,h_beta_flux,oiii_5007_flux,nii_6548_flux,nii_6584_flux,sii_6717_flux,sii_6731_flux,oi_6300_flux,class,subclass
170,299597703992600576,145.51241,0.339171,0.006385,145.2406,34.59045,4.021027,12.83837,38.72347,15.92982,11.42728,1.412207,b'GALAXY',b'STARBURST'


In [35]:
# Eliminar las filas con valores duplicados en la columna 'ra'

SDSS = SDSS.drop(indices_duplicados)

In [36]:
len(SDSS)

310346

In [37]:
indices_ra_igual_145_51241 = SDSS.index[SDSS['ra'] == 145.51241]
indices_ra_igual_145_51241

Int64Index([167], dtype='int64')

In [38]:
SDSS[100:101]

,specobjid,ra,dec,z,h_alpha_flux,h_beta_flux,oiii_5007_flux,nii_6548_flux,nii_6584_flux,sii_6717_flux,sii_6731_flux,oi_6300_flux,class,subclass
167,299595780317014016,145.51241,0.339171,0.006379,153.8173,35.59267,3.656026,13.19764,39.80714,16.52884,11.71782,1.011221,b'GALAXY',b'STARBURST'


## Conclusiones:

Se tienen los siguientes valores no repetidos por múltiples observaciones de un mismo objeto:  
* IRAS= 11.440
* SDSS= 310.346